In [ ]:
# !pip install -e linkedin_scraper_submodule

In [ ]:
# !pip install selenium

In [ ]:
import json

with open("credentials.json", "r") as file:
    credentials = json.load(file)
    email = credentials['email']
    password = credentials['password']

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Create Chrome options
chrome_options = Options()

# Disable images to speed up page loading (optional)
# chrome_options.add_argument('--blink-settings=imagesEnabled=false')

# Set user-agent (optional)
chrome_options.add_argument('--user-agent=Your User-Agent String')

# Disable GPU acceleration (useful in some headless environments)
# chrome_options.add_argument('--disable-gpu')

# # Disable notifications (optional)
chrome_options.add_argument('--disable-notifications')

# # Disable the sandbox (useful in some environments)
# chrome_options.add_argument('--no-sandbox')

# # Disable the "DevTools listening" message (optional)
chrome_options.add_argument('--disable-infobars')

# # Disable extensions (optional)
chrome_options.add_argument('--disable-extensions')

# Create a WebDriver instance with the configured options
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
from linkedin_scraper import JobSearch, actions
actions.login(driver, email, password) # if email and password isnt given, it'll prompt in terminal

In [ ]:
from pathlib import Path
from datetime import datetime
from time import sleep
import json

job_search = JobSearch(driver=driver, close_on_complete=False, scrape=False)
num_collected_jobs = 0

for cycle in range(10):
    job_listings = job_search.search("Machine Learning Engineer", num_collected_jobs+1)
    print(f"[Progress] {num_collected_jobs + len(job_listings)} jobs identified")

    for job in job_listings:
        job.scrape(False)
        sleep(10)
    num_collected_jobs += len(job_listings)
    print(f"[Progress] {num_collected_jobs} jobs collected")

    data = []
    for job in job_listings:
        datum = job.to_dict()
        datum.update({"created_date" : datetime.today().strftime("%Y-%m-%d %H:%M:%S")})
        data.append(datum)

    file_path = Path('mle.json')
    if file_path.is_file():
        print(f'The file {file_path} exists.')
        with open("mle.json", "r") as file:
            prev_data = json.load(file)
            data += prev_data

    with open("mle.json", "w") as file:
        json.dump(data, file)
    sleep(30)

print(f"Last index: {num_collected_jobs}")

In [ ]:
with open("mle.json", "w") as file:
    json.dump(data[0], file)